# FCN Training Notebook

## Import Libraries 

In [1]:
import os, sys
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.append(BASE_DIR)

from src.models.Segmentation.FCN import FCN
from src.dataset.bdd_drivable_segmentation import BDDDrivableSegmentation
from src.config.defaults import cfg
from pytorch_lightning import Trainer
from pytorch_lightning.profiler import SimpleProfiler
from pytorch_lightning.utilities.model_summary import ModelSummary
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from src.utils.DataLoaders import get_loader

## Load Datasets and DataLoaders

In [2]:
bdd_train_params = {
    'cfg': cfg,
    'stage': 'train'
}

bdd_train = BDDDrivableSegmentation(**bdd_train_params)

In [3]:
bdd_val_params = {
    'cfg': cfg,
    'stage': 'val'
}

bdd_val = BDDDrivableSegmentation(**bdd_val_params)

In [4]:
train_dataloader_args = {
    'dataset': bdd_train,
    'batch_size': 32,
    'shuffle': True,
}
train_dataloader = get_loader(**train_dataloader_args)

val_dataloader_args = {
    'dataset': bdd_val,
    'batch_size': 32,
    'shuffle': False,
}
val_dataloader = get_loader(**val_dataloader_args)

## Load Faster RCNN Model

In [5]:
fcn_model_params = {
    'cfg': cfg,
    'num_classes': len(bdd_train.cls_to_idx),
    'backbone': 'resnet101',
    'learning_rate': 1e-5,
    'weight_decay': 1e-3,
    'pretrained_backbone': True,
    'checkpoint_path': None,
    'train_loader': train_dataloader,
    'val_loader': val_dataloader
}
model = FCN(**fcn_model_params)

## Model Summary

In [6]:
ModelSummary(model, max_depth=-1)  

    | Name                                 | Type                    | Params
-----------------------------------------------------------------------------------
0   | model                                | FCN                     | 51.9 M
1   | model.backbone                       | IntermediateLayerGetter | 42.5 M
2   | model.backbone.conv1                 | Conv2d                  | 9.4 K 
3   | model.backbone.bn1                   | BatchNorm2d             | 128   
4   | model.backbone.relu                  | ReLU                    | 0     
5   | model.backbone.maxpool               | MaxPool2d               | 0     
6   | model.backbone.layer1                | Sequential              | 215 K 
7   | model.backbone.layer1.0              | Bottleneck              | 75.0 K
8   | model.backbone.layer1.0.conv1        | Conv2d                  | 4.1 K 
9   | model.backbone.layer1.0.bn1          | BatchNorm2d             | 128   
10  | model.backbone.layer1.0.conv2        | Conv2d       

## Training Section

In [ ]:
profiler = SimpleProfiler()
early_stop_callback = EarlyStopping(monitor="val_loss", patience=5, verbose=False, mode="min")
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode='min')

In [ ]:
trainer = Trainer(auto_lr_find=True, profiler=profiler, callbacks=[early_stop_callback, checkpoint_callback])

In [ ]:
trainer.fit(model)